## Exposing your tools from the MCP server

If you want to expose your tools or data so that other agents would be able to access it, then you could leverage MCP servers. Databricks already provides [managed MCP servers](https://docs.databricks.com/aws/en/generative-ai/mcp/managed-mcp#available-managed-servers), so that you do not need to start from scratch. 


These managed servers include the UC functions, vector search indexes, and Genie spaces. In our example, we will show how you can provide the agent, the UC functions that we have created in [create_tools](https://adb-984752964297111.11.azuredatabricks.net/editor/notebooks/2254899697811791?o=984752964297111#command/6040444310329070) and the vector search indexes from [pdf-rag-tool](https://adb-984752964297111.11.azuredatabricks.net/editor/notebooks/2254899697812000?o=984752964297111#command/6040444310325373) as tools stored in Databricks managed MCP servers. 

In [0]:
%pip install -U -qqqq mlflow>=3.1.1 langchain langgraph databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv databricks-feature-engineering==0.12.1
dbutils.library.restartPython()

In [0]:
%run ../_resources/01-setup

## Databricks Managed MCP

To use the UC functions and Vector Search managed MCP servers, we'd just need to add in our **agent.py** code the relevant URLs for both of them:

- **Vector search**: https://<workspace-hostname>/api/2.0/mcp/vector-search/{catalog}/{schema}
- **Unity Catalog functions**: https://<workspace-hostname>/api/2.0/mcp/functions/{catalog}/{schema}

## Update our existing Agent to add the MCP servers tools

Now that our functions and vector search indexes ready, all we have to do is to add it the managed MCP servers for both UC functions and Vector Search to our existing agent!

We'll reuse the `agent.py` and `agent_config.yaml` file: simply add the MCP configuration and our agent will add it as one of the tools available!

In [0]:
import mlflow
import yaml, sys, os
import mlflow.models

# Locate the agent_eval folder & config
agent_eval_path = os.path.abspath(os.path.join(os.getcwd(), "../02-agent-eval"))
sys.path.append(agent_eval_path)
mlflow.set_experiment(agent_eval_path+"/02.1_agent_evaluation")
conf_path = os.path.join(agent_eval_path, 'agent_config.yaml')

try:
    config = yaml.safe_load(open(conf_path))
    config["config_version_name"] = "model_with_retriever_and_mcp"

    # Inject optional MCP server tools
    config["mcp_server_tools"] = [
        {
            "type": "uc_function",
            "name": "main.default.lookup_orders"
        },
        {
            "type": "vector_search",
            "index": "main.default.customer_support_index"
        }
    ]

    yaml.dump(config, open(conf_path, "w"))
except Exception as e:
    print(f"Skipped update - ignore for job run - {e}")

model_config = mlflow.models.ModelConfig(development_config=conf_path)

## Test the agent

In [0]:
from agent import AGENT 

#Let's try our retriever to make sure we know have access to the wifi router pdf guide
request_example = "How do I restart my WIFI router ADSL-R500?"
answer = AGENT.predict({"input":[{"role": "user", "content": request_example}]})

In [0]:
with mlflow.start_run(run_name=model_config.get('config_version_name')):
  logged_agent_info = mlflow.pyfunc.log_model(
    name="agent",
    python_model=agent_eval_path+"/agent.py",
    model_config=conf_path,
    input_example={"input": [{"role": "user", "content": request_example}]},
     # Determine resources (endpoints, fonctions, vs...) to specify for automatic auth passthrough for deployment
    resources=AGENT.get_resources(),
    extra_pip_requirements=["databricks-connect"]
    )